Final script used to collect Brexit-related tweets from the Twitter Streaming API.

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import os
import json
#from CSLSauth import TwitterAuth

class TwitterAuth:
        consumer_key="OOOZHDwR4gnGyh9j9Nb6w"
        consumer_secret="tl7sjViePXuKH51rCeRwBkv4e8u0ZWPmhdjIzd9U"
        access_token="54802149-QDyrllGvqFy6gqDvti6utHwN4q9ZWDvZlihRUIzqq"
        access_token_secret="AwPuOU6KHt1Y0Y2tJxxkBhT7QN4ugxACyokuKKqBOBJsW"

# Output directory to hold json files (one per day) with tweets
outputDir = "EUref_output_streaming"
try:
        os.mkdir(outputDir)
except OSError as error:
        #Can't create directory. 
        if error.errno==17:
                #Error is that directory already exists; ignore.
                pass
        else:
                #Error is something else. Raise the error.
                raise error

##End of Settings##

class FileDumperListener(StreamListener):

        def __init__(self,filepath):
                super(FileDumperListener,self).__init__(self)
                self.basePath=filepath
                os.system("mkdir -p {}".format(filepath))

                d=datetime.today()
                self.filename = "{}-{:02d}-{:02d}.json".format(d.year,d.month,d.day)
                self.fh = open(self.basePath + "/" + self.filename,"a") #open for appending in case it already exists
                
                self.tweetCount=0
                self.errorCount=0
                self.limitCount=0
                self.last=datetime.now()

        def clean_up(self):
                self.fh.close()
        
        #This function gets called every time a new tweet is received on the stream
        def on_data(self, data):
                self.fh.write(data)
                self.tweetCount+=1
                
                #Status method prints out vitals every five minutes and also rotates the log if needed
                self.status()
                
                return True
                
        def close(self):
                try:
                        self.fh.close()
                except:
                        #Log/email
                        pass
        
        #Rotate the log file if needed.
        #Warning: Check for log rotation only occurs when a tweet is received and not more than once every five minutes.
        #This means the log file could have tweets from a neighboring period (especially for sparse streams)
        def rotateFiles(self):
                d=datetime.today()
                filenow = "{}-{:02d}-{:02d}.json".format(d.year,d.month,d.day)
                if (self.filename!=filenow):
                        print("{} - Rotating log file. Old: {} New: {}".format(datetime.now(),self.filename,filenow))
                        try:
                                self.fh.close()
                        except:
                                #Error. Should probably log/email this. Right now, ignore
                                pass
                        self.filename=filenow
                        self.fh = open("{}/{}".format(self.basePath,self.filename),"a")

        def on_error(self, statusCode):
                print("{} - ERROR with status code {}".format(datetime.now(),statusCode))
                self.errorCount+=1
        
        def on_timeout(self):
                raise TimeoutException()
        
        def on_limit(self, track):
                print("{} - LIMIT message received {}".format(datetime.now(),track))
                self.limitCount+=1
        
        def status(self):
                now=datetime.now()
                if (now-self.last).total_seconds()>300:
                        print("{} - {} tweets, {} limits, {} errors in previous five minutes.".format(now,self.tweetCount,self.limitCount,self.errorCount))
                        self.tweetCount=0
                        self.limitCount=0
                        self.errorCount=0
                        self.last=now
                        self.rotateFiles()#Check if file rotation is needed
                

class TimeoutException(Exception):
        pass


if __name__ == '__main__':

        #Create the listener
        listener = FileDumperListener(outputDir)
        OIIauth = OAuthHandler(TwitterAuth.consumer_key, TwitterAuth.consumer_secret)
        OIIauth.set_access_token(TwitterAuth.access_token, TwitterAuth.access_token_secret)
        
        terms=["#MarchForEurope", "#2ndref", "#bregretters", "#regrexit", "#bregret", "#2ndReferendum", "#SecondReferendum", "#LabourCoup", "#CorbynOut", "#WriteAPoemAboutBrexit", "#Brexitin5Words", "#BrexitFail", "#IndependentScotland", "#LibDemFightback", "#iwantedtostay", "#wearethe48", "#wearethe52", "#wearethe48percent", "#wearethe52percent", "#HopeOverFear", "#KeepCorbyn", "#itwasmyvote", "#londonstays", "#LondonIsOpen", "#LondonIndependence", "#londependence", "#ChoicesHaveConsequences", "#LetScotlandStay", "#LondonRemain", "#48percent", "#52percent", "#alwaysyes", "#LoveOurEUStaff", "#NotInOurName", "#indyref2", "#notmyvote", "#ashamedtobeBritish", "#proudtobeBritish", "#EdinEURef", "#postalvotes", "#postalvote", "#ScotlandRemain", "#ScotlandVotedRemain", "#ScottishIndependence", "#ScotlandLeave", "#Scottxit", "#Scotxit", "#dodgydave", "#camerongo", "#cameronmustgo", "#cameronresign", "#toriesout", "#stopboris", "#BorisforPM", "#MayforPM", "#GoveforPM", "#FarageforPM", "#IndependenceDay", "#WhatHaveWeDone", "#BrexitVote", "#NotInMyName", "#SaveDave", "#SaveCameron", "#itveuref", "#bbceuref", "#BBCreferendum", "#IVotedOut", "#IVotedIn", "#IDidntVote", "#EURefResult", "#EURefResults", "#IVotedLeave", "#thingsthatarewellbrexit", "#IVotedStay", "#IVotedRemain", "#iVoted", "#ReferendumDay", "#dogsatpollingstations", "#catsatpollingstations", "#BrexitDay", "#PollingStation", "#BrexitOrNot", "#otherref", "#HearOtherVoices", "#DontFuckMyFuture", "#CatsAgainstBrexit", "#CatsforBrexit", "#Cats4Brexit", "#Cats4Britain", "#PetsforBritain", "#Pets4Britain", "#Pets4Brexit", "#PetsforBrexit", "#PetsAgainstBrexit", "#Mutts4Remain", "#DogsAgainstBrexit", "#DogsforBrexit", "#Dogs4Brexit", "#Dogs4Britain", "#BrexitFacts", "#BrexitBusTour", "#referendum", "#referendumdebate", "#standupforeurope", "#beaverin", "#votebeaver", "#votin", "#EURefReady", "@BrexitE", "@pleasedontgouk", "#hugabrit", "#whyvoteleave", "#BrexitLies", "#BrexitAndChill", "#BrexitTheMovie", "#GoodbyeEU", "#LeaveEurope", "#BadBrexit", "#BrexitEffect", "#Brexiteer", "#BrexitFantasy", "#BritsDontQuit", "#CloutNotOut", "#LeadingNotLeaving", "@BeLeaveUK", "@Muslims4Britain", "@EconForBrexit", "#BankersLoveBrussels", "#fishingforleave", "#wewantourfishback", "#britinds4in", "@BritInds4IN", "@greenleaves2016", "@fishingforleave", "#EUisTheProblem", "#LeftLeave", "#LeaveAlliance", "#TheLeaveAlliance", "#2012GIR2016", "#43brokenpromises", "#AfterBrexit", "#beleave", "#BeLeaver", "#BelfastGO", "#betonbrexit", "#bettereurope", "#BetterIn", "#betterineu", "#betterineurope", "#BetterOffIn", "#BetterOffOut", "#betterout", "#bettertogether", "#bolstertheborders", "#borxit", "#Bremain", "#Brexin", "#Brexit", "#Brexit2016", "#Brexitfears", "#Brexitnow", "#brexitref", "#Brexitrisks", "#brexodus", "#brin", "#britainfirst", "#BritainIn", "#BritainOut", "#britexit", "#britin", "#brout", "#ByeByeEU", "#campaigntoremain", "#Conservatives4Britain", "#ConservativesforBritain", "#ConservativesIn", "#davesdodgydeal", "#DontWalkAway", "#englishexit", "#eudebate", "#EUDebateNI", "#eudebates", "#eudecide", "#eugood4uk", "#EUin", "#EUinorout", "#EUleave", "#EUnegotiation", "#EUout", "#EUpoll", "#EUref", "#euref16", "#euref2016", "#EUReferendum", "#eureferendum16", "#eureferendum2016", "#EURefIn", "#EUrefNI", "#EURefOut", "#EUremain", "#EUrenegotiation", "#euroref", "#eurosceptic", "#euroscepticism", "#eurosceptics", "#EUstay", "#exiteu", "#fucktheEU", "#GetBritainOut", "#goactionday", "#gobritain", "#GOGOGO", "#GOSuperSaturday", "#grassroots_out", "#grassrootsout", "#grassrootsoutgo", "#greenerin", "#incampaign", "#inorout", "#inout", "#Intogether", "#junereferendum", "#justnotintoEU", "#labourgo", "#LabourIn", "#LabourInForBritain", "#LabourLeave", "#LeadNotLeave", "#Leave", "#leavechaos", "#LeaveEU", "#leaveeuofficial", "#leavetheeu", "#leaveuk", "#LeavingEU", "#LetsTakeBackControl", "#London4Europe", "#LoveEuropeLeaveEU", "#lsebrexitvote", "#makebritaingreatagain", "#mydailybrexit", "#no2brexit", "#No2EU", "#nobrexit", "#noeu", "#notbetteroffout", "#nothankseu", "#notintoeu", "#notobrexit", "#NotoEU", "#outcampaign", "#outeu", "#pleasevoteleave", "#proeu", "#proeurope", "#proeuropean", "#projectbullshit", "#projectcheer", "#ProjectFact", "#projectfantasy", "#ProjectFear", "#projecthope", "#projectidiocy", "#projectintimidation", "#ProjectNasty", "#projectpanic", "#projectreality", "#projectsmear", "#projecttruth", "#projectwhinge", "#referendum2016", "#Remain", "#remainandgain", "#remainer", "#remainers", "#RemainEU", "#remainiac", "#remainiacs", "#Remainian", "#remainians", "#remainin", "#RemaininEU", "#saferout", "#sagedebateeu", "#sageeudebate", "#SayNo2EU", "#SayNoToEU", "#SayYes2EU", "#SayYesToEU", "#stayEU", "#stayin", "#StayinEU", "#StayinEurope", "#StoptheEU", "#StrongerIn", "#strongerout", "#studentsin", "#uk4eu", "#UKandEU", "#UKin", "#UKinEU", "#UKinEurope", "#UKinorout", "#UKout", "#UKoutEU", "#UKref", "#UKreferendum", "#UKtoStay", "#unisforeu", "#viewsoneurope", "#vote_leave", "#vote2stay", "#VoteIn", "#VoteLeave", "#VoteLeaveEU", "#VoteOut", "#VoteRemain", "#VoteStay", "#VoteToLeave", "#wewantourcountryback", "#womenforbritain", "#womenin", "#Yes2EU", "#yeseu", "#YestoEU", "#yourvoteeurope", "@vote_leave", "@LeaveEUOfficial", "@labourleave", "@Grassroots_Out", "@GetBritainOut", "@BetterOffOut", "@BrexitWatch", "@eureferendum", "@ukleave_eu", "@Stop_The_EU", "@consforbritain", "@ScienceBritain", "@BrugesGroup", "@forbritain", "@Y_Eurosceptics", "@WomenforBritain", "@StrongerIn", "@britinfluence", "@stayinEU", "@RemaininEU", "@euromove", "@Students4Europe", "@UKLabourIN", "@Scientists4EU", "@BNE_Group", "@BBCRealityCheck", "@PeoplesPledgeEU", "@lsebrexitvote", "@PollstationUK", "@Brand_EU", "@ProEuropa_GB", "@Another_Europe", "#RisksofStaying", "#takecontrol", "#WeAreEurope", "#infor", "#HealthierIn", "#Lexit", "#ImLeaveBecause", "#jun23", "#june23", "#23june", "#23june2016", "#flexcit", "#BritainInEurope", "#StudentsGO", "#BusinessGO", "#CamForEU", "@HealthierIn", "@lawyers4britain", "@weare_europe", "@weareeuropeuk", "@DemocracyMovemt", "@VeteransBritain", "@FedTrust", "@Labour4Go", "@euromovescot", "@Left4GO", "@Uni4GO", "@Scotland4GO", "@Steel4GO", "@Business4GO", "@Vapers4Britain", "@LGBT4GO", "@Nireland4GO", "@BeLeaveBritain", "@BrexitTheMovie", "@ExittheEU", "@LawyersIn4B", "@labour4europe", "@CamForEurope", "@congroup4europe", "@GlobalBritain", "@ConservativesIN"]
        print("{} - Starting stream to track {}".format(datetime.now(),", ".join(terms)))

        #Open "filter" stream tracking a geographic location
        #stream.filter(locations=[-0.530, 51.322, 0.231, 51.707])#Tweets from London

        #Open "sample" stream getting a random subset of all tweets
        #stream.sample()

        for i in range(0, 9):
                try:
                        stream = Stream(OIIauth, listener)
                        stream.filter(languages=["en"], track=terms)
                except TimeoutException:
                        stream.clean_up()
                        continue
